# Deel 2: Missing Values/Proximity Matrix
Wat zijn de grote krachten van een randomforest? Dat het relatief minder uitmaakt als bij andere algoritmes dat de data schoon is. Hieronder gaan we een paar oefeningen doen om deze missende data zo goed mogelijk te interpreteren.

In [1]:
from IPython.display import clear_output
import random
import time
random.seed(430)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore')

We laden hier de heart-disease dataset in, maar we verwijderen allerlei random waarden in de test dataset.
Wij gaan proberen om die waarden zo goed mogelijk in te vullen en een zo hoog mogelijke score te krijgen

In [2]:
df = pd.read_csv("dataset/heart.csv")
print(df.columns)
df.head(5)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df = df[['sex','age','chol','trestbps','exang', 'target']]
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), df['target'], test_size=80, random_state=99)

In [4]:
for index, row in X_test.iterrows():
    for x in range(1):
        col_to_clear = random.choice((X_test).columns)
        X_test[col_to_clear][index] = np.nan

In [5]:
print("Train dataset size:",X_train.shape[0])
print("Test dataset size", X_test.shape[0])

Train dataset size: 223
Test dataset size 80


Met deze functie kan je de accuracy testen van de missende waarden

In [6]:
clf = RandomForestClassifier(random_state=95, min_samples_split=10)
clf.fit(X_train, y_train)
clf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=95, verbose=0,
                       warm_start=False)

In [7]:
def test_model(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    return accuracy_score(y_pred, y_test)

Hieronder zie je de testdata. Hierin mist in elke row een colom met data. Het algemene idee is dat we eerst een (waarschijnlijk) slechte schatting gaan maken, en deze dan gedurende wijs te verbeteren.

In [8]:
X_test.head(10)

,sex,age,chol,trestbps,exang
137,1.0,62.0,NaN,128.0,0.0
262,1.0,NaN,282.0,123.0,1.0
43,NaN,53.0,264.0,130.0,0.0
90,1.0,NaN,255.0,124.0,0.0
32,1.0,NaN,219.0,130.0,0.0
61,1.0,54.0,309.0,NaN,0.0
72,1.0,29.0,NaN,130.0,0.0
10,1.0,54.0,NaN,140.0,0.0
240,1.0,NaN,269.0,160.0,1.0
111,NaN,57.0,126.0,150.0,0.0


In [9]:
binary_columns = ['sex','exang']

### Laten we onze baseline zetten
Maak een kopie van X_test (<b>belangrijk: niet aanpassen hier</b>)<br>
En verander alle NaN waarden in 0<br>
Test vervolgens met de test_model function welke score je krijgt.

In [10]:
# All nan values to 0
new_X_test = np.copy(X_test)
# TODO
# test_model()

### De eerste stap is een eerste simpele schatting maken
Dit doen we door voor de binaire en categoriale waardes, de meest voorkomende waarde voor die target value in te vullen.<br>
Voor de numerieke waardes pakken we het gemiddelde.<br><br>

Het is dus de bedoeling dat je de X_test invult.
Om te testen of het werkt kun je de functie test_model aanroepen.<br> 
Als je dit correct hebt gedaan zou het een score moeten opleveren van <b>0.8125</b><br>
Het is ook handig om een lijst bij te houden met alle coordinaten en kolommen die worden aangepast, zodat het duidelijk blijft welke waarden de schattingen zijn.

In [11]:
# All nan values to most common of that target
change_list = []
train_temp = X_train.join(y_train)
test_temp = X_test.join(y_test)

for index, row in test_temp.iterrows():
    for col, value in row.iteritems():
        if np.isnan(value):
            # TODO
            # Controleer eerst met de variabel binary_columns of de kolom binair is of niet
            # Vul daarna de meest voorkomende of gemmidelde in voor die target in de cell en update X_test
            # Zorg ervoor dat je in change_list bijhoudt welke kolommen en rows er zijn aangepast
X_test.head(10)

IndentationError: expected an indented block (<ipython-input-11-5a39f5114b6a>, line 13)

In [ ]:
# Controleer dan of je een betere score krijgt
test_model(clf, X_test, y_test)

# Proximity matrix
Nu we schattingen hebben ingevuld kunnen we de randomforest toepassen op de X_test<br>
Dit doen we met de apply functie. Deze functie geeft voor elke row terug bij welke leaf hij eindigt.<br>
Het idee is dat wanneer rows in dezelfde leaf eindigen, deze soortgelijk zijn.<br><br>
We houden in een proximity matrix bij welke rows gelijk zijn aan anderen<br>
Dus wanneer als voorbeeld regel 2 in dezelfde leaf node terecht komt als regel 3, Verhogen we deze cell met 1.<br>
Een proximity matrix is altijd inverted hetzelfde.

|   | 1 | 2 | 3 | 4 |
|---|---|---|---|---|
| 1 |   |   |   |   |
| 2 |   |   | 1 |   |
| 3 |   | 1 |   |   |
| 4 |   |   |   |   |

Nadat de de hele proximity matrix hebt gevuld. Doe alle getallen in de matrix gedeeld door het aantal trees.<br>
Plot hierna de matrix met plt.imshow()

In [ ]:
df_full = pd.concat([X_train, X_test])
leaves = clf.apply(df_full)
n_trees = leaves.shape[1]
proximity_matrix = np.zeros([leaves.shape[0], leaves.shape[0]])

for tree in leaves.T:
    counter += 1
    print(counter, '/' ,n_trees, end='')
    clear_output(True)
    for rowindex in df_full.index:
        for rowindex2 in df_full.index:
            if tree[rowindex] == tree[rowindex2]:
                proximity_matrix[rowindex, rowindex2] += 1
proximity_matrix /= n_trees

In [ ]:
plt.figure(figsize=[15,10])
proximity_matrix = np.zeros([leaves.shape[0], leaves.shape[0]])
plt.imshow(proximity_matrix, aspect='auto', cmap='magma')
plt.colorbar()

# Toepassen van de matrix
Nu de proximity matrix is gevuld, kan je deze gaan toepassen om de schattingen te verbeteren.<br>
### Voor binaire waardes
\begin{align}
Y = Fy \cdot (Py \div P ) \\
\end{align}
F = frequentie van yes in de dataset<br>
P = opgetelde proximities totaal of van een waarde<br>


het percentage van "ja" * het gewicht voor ja<br>
het gewicht voor ja = alle proximities voor "ja" / de totale proximities van de row

### Voor numerieke waardes
...


In [ ]:
X_test_proximized = X_test.copy()

for row in change_list:
    
    if col in binary_columns:
        # TODO
    else:
        # TODO

In [ ]:
test_model(clf, X_test_proximized, y_test)